In [27]:
import random
import numpy as np
import PDSVRP_instance
import copy

In [28]:
from importnb import Notebook

with Notebook():
    import utilities as u

metodi generali

In [29]:
def sweep_removal_operator(instance, solution, sigma):
    c_drones = u.get_drone_customers(solution[0][1])
    nbSweept = random.randint(0,int(len(c_drones) * sigma))

    grad_seed = random.randint(0, 359)
    c_drones = u.sort_by_angular(instance.coordinates, c_drones, grad_seed)
    for c in c_drones:
        if (len(solution[1]) < nbSweept):

            #inlcude tour_index calculation inside "remove_drone_customer, params: solution[0][1], c"
            tour_index = next((index for index, drone_tour in enumerate(solution[0][1]) if c in drone_tour), -1)
            solution[0][1][tour_index], solution[1] = u.remove_drone_customer(solution[0][1][tour_index], solution[1], c)
                               

    return solution

In [30]:
def random_drone_customer_removal(solution, sigma):

    c_drones = u.get_drone_customers(solution[0][1])
    q = random.randint(0,int(len(c_drones) * sigma))
    
    
    while (len(solution[1]) < q):
        customer_to_remove = random.choice(c_drones)

        c_drones.remove(customer_to_remove)
        tour_index = next((index for index, drone_tour in enumerate(solution[0][1]) if customer_to_remove in drone_tour), -1)
        solution[0][1][tour_index], solution[1] = u.remove_drone_customer(solution[0][1][tour_index], solution[1], customer_to_remove)

    return solution

In [31]:
def string_removal(instance, solution, c_average_removed, L_max): 
    l_s_max = u.max_string_length(solution[0][0], L_max)
    k_s = u.n_strings_to_remove(c_average_removed, l_s_max)
    
    c_seed = random.randint(1, instance.N - 1)

    R = []

    c_adj = u.sort_by_euclidean_distance(instance.distances, c_seed)

    for c in c_adj:
        if len(R) < k_s:
            if any(c in truck_tour for truck_tour in solution[0][0]):
                if c not in solution[1]:
                    tour_index = next((index for index, truck_tour in enumerate(solution[0][0]) if c in truck_tour), -1)
                    if tour_index not in R:
                        l = u.string_to_remove_length(l_s_max, solution[0][0][tour_index])
                        
                        solution[0][0][tour_index], solution[1] = u.remove_string(solution[0][0][tour_index], l, c, solution[1])
                        R.append(tour_index)
            elif any(c in drone_tour for drone_tour in solution[0][1]):
                if c not in solution[1]:
                    tour_index = next((index for index, drone_tour in enumerate(solution[0][1]) if c in drone_tour), -1)
                    solution[0][1][tour_index], solution[1] = u.remove_drone_customer(solution[0][1][tour_index], solution[1], c)


    return solution

In [32]:
def recreate(instance, solution, w1, w2, w3, w4, w5, gamma):
    solution[1] = u.sort_absent_customers(instance, solution, w1, w2, w3, w4, w5)
    while len(solution[1]) > 0:
        c = solution[1][0]
        pos_best = None
        
        feasible_truck_tour_positions = u.feasible_truck_tour_positions_calculation(instance, solution[0][0], c)
       
        for pos in feasible_truck_tour_positions: 
            if pos_best == None or (pos_best != None and u.cost(instance, u.insert_customer(c, pos, copy.deepcopy(solution))) < u.cost(instance, u.insert_customer(c, pos_best, copy.deepcopy(solution))) and random.random() > (1 - gamma)):
                pos_best = pos
        
        if u.is_drone_eligible(instance, solution[0][1], c):
            chosen_drone = u.largest_spatial_slack_drone(instance, solution[0][1])
            if pos_best == None or (pos_best != None and u.cost(instance, u.insert_customer(c, [1, chosen_drone, 0], copy.deepcopy(solution))) < u.cost(instance, u.insert_customer(c, pos_best, copy.deepcopy(solution))) and random.random() > (1 - gamma)):
                pos_best = [1, chosen_drone, 0]

        if pos_best == None:
            empty_truck_tour_index = next((index for index, truck_tour in enumerate(solution[0][0]) if len(truck_tour) == 0), -1)
            if empty_truck_tour_index == -1:
                solution[0][0].append([])
                empty_truck_tour_index = len(solution[0][0]) - 1
            
            pos_best = [0, empty_truck_tour_index, 0]
        
        solution = u.insert_customer(c, pos_best, solution)
        solution[1].remove(c)

    return solution

In [33]:
def ruin_and_recreate(instance, solution, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma):
    r = random.random()
    if (r < 0.5):
        solution = sweep_removal_operator(instance, solution, sigma)
    else:
        solution = random_drone_customer_removal(solution, sigma)

    solution = string_removal(instance, solution, c_average_removed, L_max)

    solution = recreate(instance, solution, w1, w2, w3, w4, w5, gamma)

    return solution


In [34]:
def local_search(instance, solution, n_nearest):

    for i in range(len(solution[0][0])):
        
        for customer in solution[0][0][i]:
        
            neighbors = u.select_nearest_neighbors(n_nearest, instance.distances[customer])
            for neighbor in neighbors:
                if (neighbor in u.get_truck_customers(solution[0][0])):
                    neighbor_index = next((j for j, node in enumerate(solution[0][0][i]) if node == neighbor), None)
                    if neighbor_index == None:
                        new_solution = u._2_opt_x(instance, copy.deepcopy(solution), customer, neighbor, i)
                        if u.cost(instance, new_solution) < u.cost(instance, solution):
                            return local_search(instance, new_solution, n_nearest)
                            
                    else: 
                        new_solution = u.relocate(instance, copy.deepcopy(solution), customer, neighbor, i)
                        if u.cost(instance, new_solution) < u.cost(instance, solution):
                            return local_search(instance, new_solution, n_nearest)
                            

                        new_solution = u.swap(instance, copy.deepcopy(solution), customer, neighbor, i)
                        if u.cost(instance, new_solution) < u.cost(instance, solution):
                            return local_search(instance, new_solution, n_nearest)
                            

                        new_solution = u._2_opt(instance, copy.deepcopy(solution), customer, neighbor, i)
                        if u.cost(instance, new_solution) < u.cost(instance, solution):
                            return local_search(instance, new_solution, n_nearest)
                            
                else:
                    new_solution = u.swap_x(instance, copy.deepcopy(solution), customer, neighbor, i)
                    if u.cost(instance, new_solution) < u.cost(instance, solution):
                        return local_search(instance, new_solution, n_nearest)
                        

            new_solution = u.shift_t(instance, copy.deepcopy(solution), customer, i)
            if u.cost(instance, new_solution) < u.cost(instance, solution):
                return local_search(instance, new_solution, n_nearest)
            

    for i in range(len(solution[0][1])):
       
        for customer in solution[0][1][i]:
            new_solution = u.shift_d(instance, copy.deepcopy(solution), customer, i)
            if u.cost(instance, new_solution) < u.cost(instance, solution):
                return local_search(instance, new_solution, n_nearest)
                

    return solution
            

In [35]:
def initial_solution_construction(instance, w1, w2, w3, w4, w5, gamma, n_nearest):
    A = [c for c in range (1, instance.N)]
    solution = recreate(instance, [[[[] for _ in range(instance.h)], [[] for _ in range(instance.D)]],A], w1, w2, w3, w4, w5, gamma)
    solution = local_search(instance, solution, n_nearest)
    return solution

In [36]:
def perturbate(instance, solution, p_min, p_max, max_unfeasible_swaps):
    p = random.randint(p_min, p_max)
    swaps_executed = 0
    unfeasible_swaps = 0
    while (swaps_executed < p and unfeasible_swaps < max_unfeasible_swaps):
        c1 = random.randint(1, instance.N - 1)
        c2 = random.randint(1, instance.N - 1)
        pos1 = u.get_position(c1, solution[0])
        pos2 = u.get_position(c2, solution[0])
        if (u.is_swap_possible(instance, solution[0], c1, pos2) and u.is_swap_possible(instance, solution[0], c2, pos1)):
            solution[0][pos1[0]][pos1[1]][pos1[2]] = c2
            solution[0][pos2[0]][pos2[1]][pos2[2]] = c1
            swaps_executed += 1
            unfeasible_swaps = 0
        else:
            unfeasible_swaps += 1

    return solution

In [37]:
def SISSRs(instance, sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma, n_nearest, delta, epsilon, iter_imp, iter_max, p_min, p_max, max_unfeasible_swaps_perturb):
    s_0 = initial_solution_construction(instance, w1, w2, w3, w4, w5, gamma, n_nearest)
    s_curr = s_0
    s_best = s_0
    iterations_without_improvement = 0
    iteration_counter = 0
    while (iteration_counter < iter_max):
        s = ruin_and_recreate(instance, copy.deepcopy(s_curr), sigma, c_average_removed, L_max, w1, w2, w3, w4, w5, gamma)
        if u.cost(instance, s) < u.cost(instance, s_curr)*(1+delta):
            s_curr = local_search(instance, s, n_nearest)
            if u.cost(instance, s_curr) < u.cost(instance, s_best):
                s_best = s_curr
                iterations_without_improvement = 0
            else:
                iterations_without_improvement += 1
        if iterations_without_improvement >= iter_imp:
            s_curr = perturbate(instance, s_curr, p_min, p_max, max_unfeasible_swaps_perturb)
            iterations_without_improvement = 0
        delta = delta * epsilon
        iteration_counter+=1
        print("Iteration: ", iteration_counter)
        print("Costo miglior soluzione: ", u.cost(instance, s_best))

    return s_best

In [38]:
SISSRs(PDSVRP_instance.PDSVRPInstance("instances/400-c-0-c.txt"), 0.3, 4.5, 4.5, 5,1,1,2,2, 0.1, 20, 0.1, 0.999975, 100, 1000, 3, 3, 9)

solution tours [[[[284, 279, 330, 148, 142, 146, 166, 380, 99, 143, 379, 387, 388, 364], [375, 373, 9, 383, 385, 376, 372, 392, 398, 395, 36, 377, 396, 389, 374, 366, 399, 397, 18, 168, 158, 123, 167, 130, 75, 109, 113, 97, 110, 114, 144, 368, 41, 78], [242, 77, 103, 101, 87, 83, 85, 96, 102, 93, 121, 381, 359, 371, 393, 361, 323, 304, 316, 292, 269, 256, 280, 238], [129, 132, 154, 131, 139, 111, 115, 92, 95, 2, 79, 71, 91, 72, 80, 281, 12, 382, 391, 365, 367, 363, 369, 297, 306], [378, 250, 239, 275, 271, 255, 258, 324, 386, 390, 160], [394, 32, 360, 124, 163, 140, 164, 127, 141, 155, 117, 150, 3, 152, 161, 134, 136, 145, 118, 112, 89, 74, 108, 105, 277, 259, 274, 270, 6, 257, 247, 283, 261, 282, 260, 264, 82, 106, 88, 84, 157], [162, 147, 156, 90, 98, 94, 104, 241, 245, 278, 86, 137, 298, 317], [289, 333, 319, 301, 276, 252, 285, 248, 273, 254, 76, 266, 253, 35, 384, 42, 400, 370, 27], [305, 303, 288, 307, 318, 287, 295, 332, 328, 7, 310, 286, 320, 331, 311, 329, 335, 322, 290, 291, 

KeyboardInterrupt: 